In [1]:
import numpy as np
np.random.seed(42)
import python_utils.python_utils.basic as basic
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import imp, pdb
fxns = imp.load_source('fxns', '/Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/code/fxns.py')

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
# constants
data_path = '/Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/data/example'
out_path = '/Users/fultonw/Documents/nonscratch/20180216_motifnet/motifnet/output/example'
train_out_path = '%s/train' % out_path
test_out_path = '%s/test' % out_path
test_chrs = [1,2]
blocksize = 1000
num_blocks = 3
create_data = False
text = False
binary = True
batch_size = 100
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

In [4]:
# create data
if create_data:
    fxns.create_shards(data_path, train_out_path, test_out_path, test_chrs, blocksize=blocksize, text=text, binary=binary, num_blocks=num_blocks)

In [5]:
# get data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

In [6]:
# print data
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    _seq_batch, _label_batch = sess.run([seq_batch, label_batch])
    print _seq_batch, _label_batch
    coord.request_stop()
    coord.join(threads)

[[0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
# define model
loss = fxns.get_logreg_model(seq_batch, label_batch, info['seq_len'], info['label_len'])
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [8]:
# train

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(tf.global_variables_initializer())
    
    i = 0
    while True:
        sess.run(optimizer)
        i += 1
        if i % 100 == 0:
            print i
    
    coord.request_stop()
    coord.join(threads)

100
200
300
400
500


KeyboardInterrupt: 